In [24]:
# from utils import *
import torchvision
import torchvision.transforms as transforms
import tensorflow as tf
import numpy as np
import datetime
# import tensorflow_datasets as tfds
import tensorflow as tf  # For tf.data
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras.applications import EfficientNetB0
from tensorflow import keras
import matplotlib.pyplot as plt
# %matplotlib inline
import numpy as np
import seaborn as sn
import pathlib
import os
from os import listdir
import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import RandomFlip, RandomRotation
from keras.applications import efficientnet, mobilenet_v3
from keras.applications import MobileNetV2
import tensorflow.keras.layers as tfl
tf.config.run_functions_eagerly(True)

# Do przeniesienia

In [2]:
from tensorflow import keras
from typing import Tuple

def load_set(path: str, label_mode: str = 'binary', batch_size: int = 16,
             image_size: Tuple[int, int] = (32, 32), val_split: float = 0.3, subset: str = 'training'):
    return keras.preprocessing.image_dataset_from_directory(
        path,
        labels='inferred',
        label_mode=label_mode,
        batch_size=batch_size,
        image_size=image_size,
        validation_split=val_split,
        subset=subset,
        shuffle=True,
        seed=1111,
    )


def load_datasets(train_path: str, valid_path: str, label_mode: str = 'binary', batch_size: int = 16,
                  image_size: Tuple[int, int] = (32, 32), val_split: float = 0.3):
    train_ds = keras.preprocessing.image_dataset_from_directory(
        train_path,
        labels='inferred',
        label_mode=label_mode,
        batch_size=batch_size,
        image_size=image_size,
        shuffle=True,
        seed=1111
    )
    valid_to_train = load_set(valid_path, label_mode, batch_size, image_size, val_split, subset='training')
    valid_ds = load_set(valid_path, label_mode, batch_size, image_size, val_split, subset='validation')
    train_ds = train_ds.concatenate(valid_to_train)
    return train_ds, valid_ds

# Load data

In [3]:
# Zuzia
# cinic_directory_train = './archive/train/'
# cinic_directory_test = './archive/test/'
# cinic_directory_valid = './archive/valid/'

#Malwina
cinic_directory_train = '../data/train/'
cinic_directory_test = '../data/test/'
cinic_directory_valid = '../data/valid/'
batch_size = 16

In [22]:
train_ds, valid_ds = load_datasets(
    train_path=cinic_directory_train, 
    valid_path=cinic_directory_valid, 
    label_mode='categorical', 
    batch_size=batch_size,
    image_size=(224, 224), 
    val_split=0.3
)

Found 90000 files belonging to 10 classes.
Found 54399 files belonging to 7 classes.
Using 38080 files for training.
Found 54399 files belonging to 7 classes.
Using 16319 files for validation.


# MobileNet

In [18]:
mobilenetv2 = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=True,
    weights='imagenet'
)

In [19]:
mobilenetv2.trainable = False

In [34]:
model = tf.keras.Sequential([
  mobilenetv2,
  # tf.keras.layers.Conv2D(128, 3, activation='relu'),
  # tf.keras.layers.Dropout(0.2),
  # tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(6, activation='softmax')
])

In [35]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), #Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [36]:
# epochs = 10
# steps_per_epoch = train_ds.n // batch_size
# validation_steps = valid_ds.n // batch_size

In [37]:
history = model.fit(train_ds, 
                              steps_per_epoch = 956,
                              epochs=10, 
                              workers=4,
                              validation_data=valid_ds, 
                              validation_steps=168)

AttributeError: 'Sequential' object has no attribute 'fit_generator'

# EfficientNet

In [12]:
inputs = layers.Input(shape=(224, 224, 3))

In [13]:
efficientnet = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
    include_top=False,
    weights='imagenet',
    input_tensor=inputs,
    classes=10,
)

In [14]:
efficientnet.trainable = False

In [15]:
x = layers.GlobalAveragePooling2D(name="avg_pool")(efficientnet.output)
x = layers.BatchNormalization()(x)
top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(10, activation="softmax", name="pred")(x)

In [16]:
model = keras.Model(inputs, outputs, name="EfficientNet")

In [17]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)

In [19]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [20]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [25]:
epochs = 5  # @param {type: "slider", min:8, max:80}
hist = model.fit(train_ds, epochs=epochs, validation_data=valid_ds)
plot_hist(hist)

Epoch 1/5


InvalidArgumentError: {{function_node __wrapped__Sub_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [1280] vs. [0] [Op:Sub] name: 

In [26]:
%tensorboard --logdir logs/fit

UsageError: Line magic function `%tensorboard` not found.


# Efficient 2

In [27]:
efficientnet = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

In [28]:
efficientnet.trainable = False

In [30]:
model_efficientnet = tf.keras.Sequential([
    efficientnet,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [33]:
model_efficientnet.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01), #Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
history = model_efficientnet.fit(train_ds, 
      epochs=5, 
      validation_data=valid_ds)

Epoch 1/5
 871/8005 ━━━━━━━━━━━━━━━━━━━━ 12:53:25 7s/step - accuracy: 0.6855 - loss: 1.0771